In [1]:
from streamlit_app import *

In [2]:
import datetime
from logging import exception
from typing import List

import gspread
import numpy as np
import pandas as pd
import plotly.express as px
import streamlit as st
from google.oauth2 import service_account
import matplotlib.colors as mcolors

In [3]:
sheet_url = st.secrets["private_gsheets_url"]
df = get_worksheet(sheet_url, 0)

2022-02-05 21:18:51.073 
  command:

    streamlit run /home/wotashu/.cache/pypoetry/virtualenvs/streamlit-cheki-5kyQcRg0-py3.9/lib/python3.9/site-packages/ipykernel_launcher.py [ARGUMENTS]


In [4]:
cheki_df = get_worksheet(sheet_url, 0)
idol_df = get_worksheet(sheet_url, 1)
venue_df = get_worksheet(sheet_url, 2)

In [6]:
cheki_df['checki_id'] = cheki_df.index

In [10]:
name_cols = [i for i in cheki_df.columns if "Name" in i]

In [16]:
cheki_df['idol_count'] = cheki_df[name_cols].replace("", np.nan).count(axis=1)

In [18]:
cheki_df.replace("", np.nan)

,Note,Location,Date,Name1,Name2,Name3,Name4,Name5,Name6,Name7,Name8,checki_id,idol_count
1,NaN,Bellesalle Shibuya Garden,2013-05-05,矢島舞美@℃-ute,中島早貴@℃-ute,鈴木愛理@℃-ute,岡井千聖@℃-ute,萩原舞@℃-ute,NaN,NaN,NaN,1,5
2,NaN,Best Western Plus Silverdale Beach Hotel,2014-06-21,AIRI@Aither,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,1
3,NaN,Best Western Plus Silverdale Beach Hotel,2014-06-21,SAKURA@Aither,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3,1
4,NaN,Best Western Plus Silverdale Beach Hotel,2014-06-21,YUUKA@Aither,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4,1
5,NaN,Best Western Plus Silverdale Beach Hotel,2014-06-21,木村美咲@ラストアイドル,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4358,NaN,Club Asia,2022-02-05,松島朱里@メルティハート,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4358,1
4359,NaN,Club Asia,2022-02-05,涼乃みほ@メルティハート,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4359,1
4360,NaN,Club Asia,2022-02-05,涼乃みほ@メルティハート,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4360,1
4361,NaN,Club Asia,2022-02-05,侑之りせ@メルティハート,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4361,1
